In [1]:
import numpy as np
import sklearn.datasets as dsets
from sklearn import metrics, tree
from sklearn.model_selection import train_test_split

In [2]:
class DTree(object):
    def __init__(self, max_depth = 5):
        self.max_depth = max_depth
    
    def fit(self, X, y):
        self.initlialize(X, y)
        i = 0
        while i < self.next_node:
            if self.depth[i] <= self.max_depth and self.node[i].size > 1:
                self.split_node(i)
            i += 1
        
    def initlialize(self, X, y):
        self.X = X
        self.y = y
        self.next_node = 0
        self.node = []
        self.left_node = []
        self.right_node = []
        self.depth = []
        self.index = []
        self.t = []
        self.init_new_node(np.arange(0, y.shape[0], 1, dtype=int), 0)
        
    def init_new_node(self, indexes, depth):
        self.node += [indexes]
        self.left_node += [-1]
        self.right_node += [-1]
        self.index += [-1]
        self.t += [-1]
        self.depth += [depth]
        self.next_node += 1
        
        
    def split_node(self, ind):
        x = self.X[self.node[ind]]
        y = self.y[self.node[ind]]
        x_temp = x.T
        m_probas = []
        for probas in x_temp:
            avb = np.array(list(set(list(np.sort(probas, axis = 0)))))
            avb = (avb[:-1] + avb[1:]) / 2
            m_probas += [avb]
        
        MSE = 1e308
        pr = -1
        t = -1
        for i, probas in enumerate(m_probas):
            for proba in probas:
                new_MSE = self.try_split(y, x[:,i] <= proba)
                if new_MSE < MSE:
                    pr = i
                    t = proba
                    MSE = new_MSE
        if pr == -1:
            return
        left = []
        right = []
        for i, one in enumerate(x):
            if one[pr] <= t:
                left += [self.node[ind][i]]
            else:
                right += [self.node[ind][i]]
        self.index[ind] = pr
        self.t[ind] = t
        self.left_node[ind] = self.next_node
        self.init_new_node(np.array(left), self.depth[ind] + 1)
        self.right_node[ind] = self.next_node
        self.init_new_node(np.array(right), self.depth[ind] + 1)
        
    
    def try_split(self, y, x):
        y1 = y[x]
        y2 = y[x == False]
        res = 0
        res += ((y1 - y1.mean(0)) ** 2).sum(0)
        res += ((y2 - y2.mean(0)) ** 2).sum(0)
        return res
    
    def predict_one(self, X):
        ind = 0
        while self.index[ind] > -1:
            if X[self.index[ind]] <= self.t[ind]:
                ind = self.left_node[ind]
            else:
                ind = self.right_node[ind]
        return self.y[self.node[ind]].mean(0)
    
    def predict(self, X):
        if len(X.shape) > 1:
            res = []
            for x in X:
                res += [self.predict_one(x)]
            return np.array(res)
        else:
            return predict_one(X)

In [3]:
X, y = dsets.load_boston(return_X_y=True)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=24)
mytree = DTree(max_depth=7)
sktree = tree.DecisionTreeRegressor(max_depth=7)

In [4]:
sktree.fit(X_train, y_train)
mytree.fit(X_train, y_train)

In [5]:
print "errors"
print "my tree: %s"%metrics.mean_squared_error(y_test, mytree.predict(X_test))
print "sklearn: %s"%metrics.mean_squared_error(y_test, sktree.predict(X_test))
print "mean: %s"%metrics.mean_squared_error(y_test, np.array([y_train.mean(0)]*y_test.size))

errors
my tree: 27.5961880017
sklearn: 15.3697106735
mean: 68.6418202175
